<a href="https://colab.research.google.com/github/ZPavlo/GazeLandmarksHourGlass/blob/main/eye_landmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DATA = 'gdrive/MyDrive/unityeye/'

In [ ]:
import os, sys

!git clone https://github.com/ZPavlo/GazeLandmarksHourGlass 
sys.path.append("GazeLandmarksHourGlass")

In [ ]:
!pip install progress

In [8]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from model.hourglass import HourglassNet

NUM_STACKS = 3
NUM_BLOCKS = 4
NUM_CLASSES = 17
print("==> creating model: stacks={}, blocks={}".format(NUM_STACKS, NUM_BLOCKS))
model = HourglassNet(num_stacks=NUM_STACKS, num_blocks=NUM_BLOCKS, num_classes=NUM_CLASSES)
model = model.to(device)

In [10]:
from losses import JointsMSELoss
from utils.optimizer import get_optimizer

LR = 0.005

criterion = JointsMSELoss().to(device)
optimizer = get_optimizer("adam", model, lr=LR)

In [ ]:
from utils.data_preprocessing import EyeLandmarksDataset, TrainDataset, TestDataset
from torch.utils.data import DataLoader

unity_eye = EyeLandmarksDataset(ROOT_DATA, load_full=True)
train_dataset = TrainDataset(unity_eye, data_config={
    "max_shift": (5, 7),
    "delta_scale": 0.4,
    "max_rotation_angle": 0.5,
    "image_size": (128, 96),
    "line_count": 2,
    "down_up_scale": 0.4,
    "sigma_head_map": 35.0,
})

test_dataset = TestDataset(unity_eye, {
    "line_count": 0,
    "image_size": (128, 96),
})

In [12]:
from train import train, validate
from utils.tools import adjust_learning_rate, save_checkpoint
lr = LR

lrs = []
train_losses = []
valid_losses = []
train_acc_list = []
valid_acc_list = []
start_epoch = 0
best_acc = 0.0

In [13]:
BATCH_SIZE = 16
NUM_WORKERS = 1

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
import numpy as np
import torch
import os

EPOCH_SIZE = 5
SNAPSHOT = 1
CHECKPOINT_PATH = r"/content/gdrive/MyDrive/exp4"

if not os.path.isdir(CHECKPOINT_PATH):
    os.mkdir(CHECKPOINT_PATH)

difficult = np.linspace(0.15, 0.65, EPOCH_SIZE)

SCHEDULE = [2, 3, 4]
GAMMA = 0.8

for epoch in range(start_epoch, EPOCH_SIZE):
    lr = adjust_learning_rate(optimizer, epoch, lr, SCHEDULE, GAMMA)
    print('\nEpoch: %d | LR: %.8f' % (epoch + 1, lr))

    train_dataset.set_difficult(difficult[epoch])
    test_dataset.set_difficult(difficult[epoch])
    # train for one epoch
    train_loss, train_acc, best_acc = train(train_dataloader, model, criterion, optimizer, train_size=1000,
                                            print_step=10, save_step=550, checkpoint_path=CHECKPOINT_PATH,
                                            best_acc=best_acc)

    # evaluate on validation set
    valid_loss, valid_acc, best_acc = validate(test_dataloader, model, criterion, print_step=10, best_acc=best_acc)

    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc': max(valid_acc, best_acc),
        'optimizer': optimizer.state_dict(),
    }, valid_acc > best_acc, checkpoint=CHECKPOINT_PATH, snapshot=SNAPSHOT)


In [ ]:
print(train_losses)
print(valid_losses)
print(train_acc_list)
print(valid_acc_list)